Authors
- Shawn Fan
- Artur Dox
- Ang Li
- Holly Whitfield
- Ramon Vinas Torne

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import anndata
from metrics import *
from de import *
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
data_file = '../datasets/data/1gene-norman.h5ad'
adata = anndata.read_h5ad(data_file)
adata_control = adata[adata.obs['perturbation_name'] == 'control']
control_mean = np.array(adata_control.X.mean(axis=0))[0]

# Select subset of perturbations for testing purposes
adata_subset = adata[adata.obs['perturbation_name'].isin(['FOXA1', 'FOXA3'])]

# Simulate split (ground truth cells, predicted cells)
idxs_gt, idxs_pred = train_test_split(np.arange(len(adata_subset)), test_size=0.5)
adata_gt = adata_subset[idxs_gt]
adata_pred = adata_subset[idxs_pred]

In [4]:
# Run DE analysis
de_genes_gt = get_de_genes(adata_control, adata_gt,
                           method = "wilcoxon",
                           top_k = 100,
                           groupby_key = 'perturbation_name')
de_genes_pred = get_de_genes(adata_control, adata_pred,
                             method = "wilcoxon",
                             top_k = 100,
                             groupby_key = 'perturbation_name')

/home/vinas/scEvalsJam/metrics/de.py:41: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  anndata_x.obs["group"] = "control"
/home/vinas/scEvalsJam/metrics/de.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  anndata_y.obs["group"] = "perturbation"
/home/vinas/scEvalsJam/metrics/de.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  anndata_y.obs["group"] = "perturbation"


In [8]:
### --- Calculate metrics per scenario
unique_perturbations = adata_subset.obs['perturbation_name'].unique()

## Calculate metrics
results_list = [calc_metrics(adata_gt, adata_pred, adata_control,
                             pert, metric_dict, 
                             de_genes_gt, de_genes_pred, 
                             de_subset = None) for pert in tqdm(unique_perturbations)]
## Build dataframe
results_df = pd.DataFrame(results_list,
                          index=[i for i in unique_perturbations], 
                          columns=list(metric_dict.keys())+["Jaccard_de_up", "Jaccard_de_dn"])

## Remove redundant metrics
redundant_column_mask = results_df.columns.str.contains('ec') & results_df.columns.str.contains('mse|mae|euclidean_distance|bhattacharyya_distance')
results_df = results_df.loc[:, ~redundant_column_mask]

  0%|                                                     | 0/2 [00:00<?, ?it/s]An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/vinas/scEvalsJam/metrics/metrics.py:65: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - np.mean(intersection / union)
/home/vinas/miniconda3/envs/scpert/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
 50%|██████████████████████▌                      | 1/2 [00:13<00:13, 13.19s/it]/home/vinas/scEvalsJam/metrics/metrics.py:65: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - np.mean(intersection / union)
/home/vinas/miniconda3/envs/scpert/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:66: UserWarning: Clustering metrics expects discrete value

In [9]:
results_df

,mse,mae,ks_test_distance,edistance,cosine_distance,pearson_distance,euclidean_distance,classifier_proba,kendalltau_distance,spearman_distance,wasserstein,sym_kldiv,bhattacharyya_distance,jaccard_index,F1_score,adjusted_rand_index,mutual_information,fowlkes_mallows_index,Jaccard_de_up,Jaccard_de_dn
FOXA1,0.000324,0.010741,0.014995,12.313578,0.063507,0.078068,2.481288,0.426261,4.220650e+07,0.308070,2583.874889,9.008680e+11,0.000056,NaN,0.0,0.0,0.973725,0.193627,0.459854,0.492537
FOXA3,0.000526,0.013790,0.019283,20.005765,0.064992,0.077410,3.162733,0.395555,4.494570e+07,0.351932,2599.142979,1.557142e+12,0.000092,NaN,0.0,0.0,0.976233,0.185156,0.398601,0.481481
